# RBM Hyperparameter Tuning

This notebook demonstrates how to tune the hyperparameters of the Restricted Boltzmann Machine (RBM) algorithm using GridSearchCV and compares the performance of tuned vs untuned RBM against random recommendations.

### Setup and Imports

First, let's import the necessary libraries and set up our random seeds for reproducibility.

In [1]:
from recsys.MovieLens import MovieLens
from RBM import RBM
from surprise import NormalPredictor
from recsys.Evaluator import Evaluator
from surprise.model_selection import GridSearchCV

import random
import numpy as np
import pandas as pd

np.random.seed(0)
random.seed(0)

### Load Data

Load the MovieLens dataset and prepare it for evaluation.

In [2]:
lens, ratings_data, rankings = MovieLens.load()

### Grid Search for Best Parameters

We'll use GridSearchCV to find the optimal hyperparameters for the RBM algorithm. We'll search over:
- Hidden dimensions (10, 20)
- Learning rate (0.01, 0.1)

The search will use 3-fold cross-validation and evaluate using both RMSE and MAE metrics.

In [3]:
print("Searching for best parameters...")
param_grid = {
    'hidden_dim': [20, 10],
    'learning_rate': [0.1, 0.01]
}
gs = GridSearchCV(RBM, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(ratings_data)

# best RMSE score
print("Best RMSE score attained: ", gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print("Best parameters:", gs.best_params['rmse'])

### Initialize Evaluator and Add Algorithms

Now we'll create an evaluator and add three algorithms to compare:
1. RBM with tuned parameters (using the best parameters found by GridSearchCV)
2. RBM with default parameters
3. Random recommendations (baseline)

In [4]:
evaluator = Evaluator(ratings_data, rankings)

# Add tuned RBM
params = gs.best_params['rmse']
rbm_tuned = RBM(hidden_dim=params['hidden_dim'], 
                       learning_rate=params['learning_rate'])
evaluator.add_algorithm(rbm_tuned, "RBM - Tuned")

# Add untuned RBM
rbm_untuned = RBM()
evaluator.add_algorithm(rbm_untuned, "RBM - Untuned")

# Add random recommendations as baseline
Random = NormalPredictor()
evaluator.add_algorithm(Random, "Random")

### Evaluation results

In [ ]:
results = evaluator.evaluate(top_n_metrics=True, coverage_threshold=4.0)

results.to_df()

### Sample Recommendations

Generate and display some sample recommendations using the evaluated algorithms. 

This will help us qualitatively assess the differences between tuned and untuned RBM recommendations.

In [6]:
samples = evaluator.sample_top_n_recs(uid=85)

for algorithm, recs in samples.items():
    print(f"{algorithm}")
    movie_names = lens.get_movie_names(recs)
    for movie_name in movie_names:
        print(f"  {movie_name}")